# Overview

No sure why AWQ cannot work in this time in this notebook.

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [2]:
%%capture --no-stder
# There are several libraries for quantizing models with the AWQ algorithm
# like llm-awq, here we use autoawq
!pip install triton==2.2.0
!pip install autoawq==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 56.4 MB/s eta 0:00:00:00:0100:01


In [3]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


It is a quantization method similar to GPTQ. There are several differences between AWQ and GPTQ as methods but the most important one is that AWQ assumes that not all weights are equally important for an LLM's performance. In other words, there is a small fraction of weights that will be skipped during quantization which helps with the quantization loss.

Loading an AWQ-quantized model automatically sets other weights to fp16 by default for performance reasons. If you want to load these other weights in a different format, use the `torch_dtype` parameter

In [4]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = "facebook/opt-125m"
quant_path = "opt-125m-awq"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version":"GEMM"}

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True, device_map='auto', trust_remote_code=True)

2024-02-16 05:54:29.328236: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 05:54:29.328339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 05:54:29.451853: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

LICENSE.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

OSError: Error no file named model.safetensors found in directory /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/27dcfa74d334bc871f3234de431e71c6eeba5dd6.

In [ ]:
!rm -rf /root/.cache huggingface

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer

In [ ]:
# Quantize
model.quantize(tokenizer, quant_config=quant_config)
model

In [ ]:
quant_config

In [ ]:
from transformers import AwqConfig, AutoConfig

# modify the config file so that it is compatible with transformers integration
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# the pretrained transformers model is stored in the model attribute + we need to pass a dict
model.model.config.quantization_config = quantization_config
# a second solution would be to use Autoconfig and push to hub (what we do at llm-awq)


# save model weights
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

In [ ]:
from huggingface_hub import HfApi

os.environ["AWQ_MODEL"]="aisuko/opt-125m-awq"

api = HfApi()
api.create_repo(repo_id=os.getenv('AWQ_MODEL'),exist_ok=True,repo_type="model")

api.upload_folder(
    folder_path="opt-125m-awq",
    repo_id=os.getenv('AWQ_MODEL'),
    repo_type="model",
)
tokenizer.push_to_hub("aisuko/opt-125m-awq")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Use the device_map parameter to specify where to place the model
model=AutoModelForCausalLM.from_pretrained(
    os.getenv("AWQ_MODEL"),
#     attn_implementation="flash_attention_2",
    device_map="auto",
)

model.get_memory_footprint()

In [ ]:
model.config.quantization_config

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(os.getenv('AWQ_MODEL'))
model = AutoModelForCausalLM.from_pretrained(os.getenv('AWQ_MODEL')).to(0)

text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = model.generate(**inputs, max_new_tokens=5)
print(tokenizer.decode(out[0], skip_special_tokens=True))

# Credit

* https://colab.research.google.com/drive/1HzZH89yAXJaZgwJDhQj9LqSBux932BvY#scrollTo=RWIh6iUR_i5P